# Lecture 8: PyMC3 
## ——基于Python的概率编程框架

## Instructor： 胡传鹏（博士）[Dr. Hu Chuan-Peng]

### 南京师范大学心理学院[School of Psychology, Nanjing Normal University]

# Part 1: Probabilistic Programming Languages (PPL) and PyMC3

## Probabilistic Programming Languages (PPL)

回顾一下我们本学期第一节课和后续课程的内容：贝叶斯模型是一种更好的方法。

贝叶斯方法有一个显著的优势：每次使用贝叶斯方法我们都是得到一个分布并进行推断，这个分布也代表了一个有关世界结构的模型（小世界与大世界）。

### Deterministic vs Probabilistic: 
确定性编程: 精确的模型, 相同的输入总是产生相同的输出。

例如：

In [1]:
sum([1,2])

3

解决概率问题需要概率性的函数与功能，因此也产生了Probabilistic Programming Languages (PPL)，即概率编程语言。

概率编程语言与非概率编程语言/确定性编程不同
- 概率编程语言允许使用者可以使用普通计算句语句来指定一个随机过程，简化了概率模型的开发
- 概率程序从建模的联合分布中生成样本，并在给定模型的情况下自动进行推断

概率编程语言执行流程
- 概率编程语言通过用计算机语言建构一个模型

- 这个建立的模型生成观察值

- 内置的推断方法将观测值返回可能生成观察值的参数分布

- 之后还可以通过检验后验概率生成的数据对模型进行评估



![Image Name](https://cdn.kesci.com/upload/image/rk4wg2rtm.jpg?imageView2/0/w/960/h/960)

通俗地说，PPL就是一种规范的语法，来描述带有概率的推理过程（生成过程）。

### 常见的概率编程语言

- **Stan**：主要来自哥伦比亚大学的Andrew Gelman、Bob Carpenter和其他人开发的PPL。它有自己的模型规范语言，然后将其转换为C++代码，最后编译为机器代码。良好的R和python接口可用（rstan, PyStan）。更多信息查看：https://mc-stan.org/
- **PyMC3**：一个广泛使用的带有Theano后端的Python开源PPL，由John Salvatier、Thomas Wiecki、Chris Fonnesbeck和其他贡献者开发。更多信息查看：https://docs.pymc.io/en/v3/
- **Edward**和**Edward2**：是一个用于概率建模、推断的Python库。由Google的Dustin Tran、哥伦比亚大学的David Blei 和其他合作者的基于TensorFlow的开发的PPL。更多信息查看：https://github.com/google/edward2
- **Pyro**：由Uber AI Labs创建的带有PyTorch后端的PPL。更多信息查看：https://pyro.ai/
- JAGS 
- ...

![常见的ppl](https://cdn.kesci.com/upload/image/rk8m3hh1o1.png?imageView2/0/w/960/h/960)

## PyMC3
PyMC是Python的概率编程库，允许用户使用简单的Python API构建贝叶斯模型，并使用马尔可夫链蒙特卡洛（MCMC）方法拟合它们。
![Image Name](https://cdn.kesci.com/upload/image/rk56d0ypat.png?imageView2/0/w/960/h/960)

PyMC目前是第四代，名称回归到PyMC，在此前比较重要的有PyMC 2，其主要使用Gibbs算法和MH算法。

相比于其他概率编程语言，PyMC3有以下几种优势：
- 基于python强大的社区，PyMC3可以包括很多新的算法
- 相比于其他概率编程语言，Pymc写模型更加容易（定义模型时更加符合我们平时使用的语言）
- 速度相对较快


基于所有参数的最小有效样本量 (ESS) 的非常初步的结果。每个模型都运行了10000次迭代的单个链（带有1000次预热）。

所有PPL都在Colab中运行。
![Image Name](https://cdn.kesci.com/upload/image/rk8m6lwp5o.png?imageView2/0/w/960/h/960)

source: https://theorashid.github.io/post/ppl-benchmark-help/

## Install PyMC3

PyMC3需要使用使用theano库来进行快速的数值计算，但是该库已经停止继续维护了，所以安装theano库时可能会出现一些问题。解决方法如下：

```
# windows电脑在终端输入下面代码
python3 -m pip uninstall theano

python3 -m pip uninstall theano-pymc

python3 -m pip uninstall pymc3

python3 -m pip install pymc3
```

```
#mac电脑直接使用pip后面的内容即可
pip uninstall theano

pip uninstall theano-pymc

pip uninstall pymc3

pip install pymc3
```

安装问题可以访问链接链接：https://github.com/pymc-devs/pymc/issues/4479

安装完成后可以按照下方的代码进行导入：

In [2]:
#基本的库和模块
import numpy as np
from scipy import stats
import pandas as pd
from patsy import bs, dmatrix
import matplotlib.pyplot as plt

#贝叶斯模型探索分析
import arviz as az

# 概率编程语言
import pymc3 as pm

#计算后端
import theano
import theano.tensor as tt

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'


## Part 2: PyMC3的语法[The syntax of PyMC3]

### 实例：
社交网站的使用可能在知识共享和个体创新行为中起到了关键作用，而个体在使用社交网站时往往能获得在线社会支持，可以帮助个体提高对自身创新行为产生的能力和信心的评估——即创新自我效能感。

假设我们将探究大学生的社交网站使用和创新的自我效能感对个体创新行为的关系。

`SNS_t`:  社交网站使用强度；
`CSES_t`: 创新自我效能感；
`EIB_t`: 创新行为

数据来源：郑元瑞, 谢嘉敏, 李鹏. (2022) 社交网站使用强度对大学生创新行为的影响：一个有调节的中介模型. 心理技术与应用, 10(8), 483-491.

In [3]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import arviz as az
# sklearn是用于机器学习的包，precessing是其中的预处理模块，StandardScaler函数的作用是将变量标准化
from sklearn.preprocessing import StandardScaler
# mpl_toolkits.mplot3d是用于三维画图的包，Axes3D用于画三维图
from mpl_toolkits.mplot3d import Axes3D

In [4]:
np.random.seed(123)  # 随机数种子，确保随后生成的随机数相同
data = pd.read_csv("/home/mw/input/data9464/clean.csv") # 读取数据
data['SNS_t'] = (data['SNS_t'] - data['SNS_t'].mean()) / data['SNS_t'].std()     # 将变量进行标准化
data['CSES_t'] = (data['CSES_t'] - data['CSES_t'].mean()) / data['CSES_t'].std() # 将变量进行标准化
data['EIB_t'] = (data['EIB_t'] - data['EIB_t'].mean()) / data['EIB_t'].std()     # 将变量进行标准化

In [19]:
data.shape

(1014, 23)

In [18]:
data.tail()

,Unnamed: 0,id,IP,sex,grade,residence,school,SNS_t,SES_t,CSES_t,...,female,male,freshman,sophomore,junior,senior,rural,city,private,public
1009,1009,1010,广西,female,senior,city,private,-0.267466,0.323852,0.988921,...,1,0,0,0,0,1,0,1,1,0
1010,1010,1011,广东,female,junior,rural,public,-0.080304,-0.125069,-1.573512,...,1,0,0,0,1,0,1,0,0,1
1011,1011,1012,广西,female,senior,rural,private,1.393686,0.997233,0.988921,...,1,0,0,0,0,1,1,0,1,0
1012,1012,1013,广西,male,senior,rural,public,0.258998,-0.125069,0.348312,...,0,1,0,0,0,1,1,0,0,1
1013,1013,1014,云南,male,freshman,rural,private,-0.588694,1.670615,-0.719368,...,0,1,1,0,0,0,1,0,1,0


In [5]:
#创建一个画布fig1，该画布有两幅图ax1,ax2，画布尺寸为（10，4）
fig1,ax1 = plt.subplots(1,2,figsize=(10,4))

#在第一张图的第一个panel上画散点图
ax1[0].scatter(data['SNS_t'],data['EIB_t'] )
ax1[0].set_xlabel('SNS_t')
ax1[0].set_ylabel('EIB_t')

#在第一张图的第二个panel上画散点图
ax1[1].scatter(data['CSES_t'],data['EIB_t'] )
ax1[1].set_xlabel('CSES_t')
ax1[1].set_ylabel('EIB_t')

#在第二张图（三维空间）画散点图
fig2 = plt.figure(2)
ax2 = Axes3D(fig2)
ax2.scatter(data['SNS_t'],data['CSES_t'],data['EIB_t'] )
ax2.set_xlabel('SNS_t')
ax2.set_ylabel('CSES_t')
ax2.set_zlabel('EIB_t')
plt.show()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Axes3D(fig) adding itself to the figure is deprecated since 3.4. Pass the keyword argument auto_add_to_figure=False and use fig.add_axes(ax) to suppress this warning. The default value of auto_add_to_figure will change to False in mpl3.5 and True values will no longer work in 3.6.  This is consistent with other Axes classes.
  app.launch_new_instance()


<Figure size 720x288 with 2 Axes>

<Figure size 432x288 with 1 Axes>

可使用线性模型来探索`创新自我效能感`和`社交网站使用强度`对`个体创新行为`的关系。我们可以使用`PyMC3`来建立该方程.

在PyMC3中，一个简单的线性模型：

1. 通过建立线性模型的概率表达形式来构建模型。

2. 通过PyMC3对后验进行采样

3. 通过Arviz对结果进行展示，辅助统计推断

线性模型的一般形式：

$y_i = a + b_1 x_1 + b_2 x_2 + \epsilon$

$y_i = \hat{y} + \epsilon$

线性模型可以用概率的形式进行表达

$y \sim Normal(\mu_i, sigma)$      ->      y $\sim$ Normal(mu,sigma)

$\mu_i  = \alpha+\beta_1 * x1 +\beta_2 * x_2$      ->      mu = alpha + $beta_1$  * $x_1$  + $beta_2$   * $x_2$

$\alpha \sim Normal(0,10)$      ->      a $\sim$ Normal(mu,sigma)

$\beta_{i}  \sim Normal(0,10)$      ->      $b_i$ $\sim$ Normal(mu,sigma)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)





接下来我们将展示如何用代码将这一形式进行表达。

#### 模型构建

In [6]:
#首先，载入pymc3这个包，将其命名为pm
import pymc3 as pm
# 在pymc3中，pm.Model()定义了一个新的模型对象，这个对象是模型中随机变量的容器
# 在python中，容器是一种数据结构，是用来管理特殊数据的对象
# with语句定义了一个上下文管理器，以 linear_model 作为上下文，在这个上下文中定义的变量都被添加到这个模型
with pm.Model() as linear_model:
    # 先验分布: alpha, beta, sigma这三个参数是随机变量
    alpha = pm.Normal('alpha',mu=0,sd=10)
    beta = pm.Normal('beta',mu=0,sd=10, shape=2)  # shape 用来表示同样的分布的参数的数量
    sigma = pm.HalfNormal('sigma',sd=1)

    # x1,x2为自变量，是之前已经载入的数据
    x1 = pm.Data("x1", data['SNS_t'])
    x2 = pm.Data("x2", data['CSES_t'])

    # 线性模型：mu是确定性随机变量，这个变量的值完全由右端值确定
    mu = pm.Deterministic("mu", alpha + beta[0]*x1 + beta[1]*x2) 

    # Y的观测值，这是一个特殊的观测随机变量，表示模型数据的可能性。也可以表示模型的似然，通过 observed 参数来告诉这个变量其值是已经被观测到了的，不会被拟合算法改变
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=data['EIB_t'])


该模型结构可以用可视化的形式进行表达。

可以通过PyMC3自带的可视化工具将模型关系可视化。

x1，x2为自变量。

参数 $\alpha$ 是线性模型的截距，而 $\beta1$，$\beta2$ 是斜率。

参数$sigma$是残差，因变量为$y$。

模型图展示了各参数通过怎样的关系影响到因变量。

In [7]:
pm.model_to_graphviz(linear_model)

#### 模型拟合

构建模型之后，我们可以进行模型拟合

获得模型中未知变量的后验估计。考虑两个方法：
（1）使用优化方法找到参数的极大后验估计点(maximum a posteriori(MAP))

我们也可以使用极大后验估计 MAP(pm.find_MAP)来进行优化。

（2）使用MCMC采样方法获得后验分布来计算。

在本例中，我们使用MCMC来进行采样，函数为pm.sample(),

In [8]:
#采样过程仍在该容器中进行
with linear_model :
    # 使用mcmc方法进行采样，draws为采样次数，tune为调整采样策略的次数，这些次数将在采样结束后被丢弃，
    # target_accept为接受率， return_inferencedata=True为该函数返回的对象是arviz.InnferenceData对象
    # chains为我们采样的链数，cores为我们的调用的cpu数，多个链可以在多个cpu中并行计算，我们在和鲸中调用的cpu数为2
    trace = pm.sample(draws = 2000, tune=1000, target_accept=0.9,chains=2, cores= 2,return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 2_000 draw iterations (2_000 + 4_000 draws total) took 6 seconds.


#### 参数的后验分布
这里的模型分析结果展示了各参数的分布(后验)情况

ArviZ的plot_trace会展示左右两幅图:

左图描述了采样的各个马尔可夫链参数的后验分布，左图的横坐标为后验分布的取值范围，纵轴为概率密度。如果有两个参数在同一幅图中，就会用不同颜色的曲线表示

右图描述了采样的各个马尔可夫链的参数的采样情况，右图的横坐标为每次采样次数，纵坐标为每一次采样的值

In [9]:
# 绘制特定参数的采样情况，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.plot_trace(trace,var_names=['alpha','beta','sigma'])

array([[<AxesSubplot:title={'center':'alpha'}>,
        <AxesSubplot:title={'center':'alpha'}>],
       [<AxesSubplot:title={'center':'beta'}>,
        <AxesSubplot:title={'center':'beta'}>],
       [<AxesSubplot:title={'center':'sigma'}>,
        <AxesSubplot:title={'center':'sigma'}>]], dtype=object)

<Figure size 864x432 with 6 Axes>

ArviZ的plot_forest会展示森林图

该图会展示不同后验分布的94%的可信区间，横轴为后验分布的取值范围，纵轴为不同参数。

对于每一个参数，粗线展示了采样的各马尔可夫链的94%的可信区间，细线展示了参数的取值范围。

若r_hat=True，右图会展示r_hat的值，该值通过比较模型参数的链间和链内估计值反映了采样的收敛性，r-hat值越接近1则收敛情况越好。

In [10]:
# 绘制特定参数的森林图，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.plot_forest(trace,var_names=['alpha','beta','sigma'],r_hat=True)

array([<AxesSubplot:title={'center':'94.0% HDI'}>,
       <AxesSubplot:title={'center':'r_hat'}>], dtype=object)

<Figure size 576x489.6 with 2 Axes>

ArviZ的plot_posterior绘制了参数的后验分布图

该图的横轴为后验分布值，横线为94%可信区间，曲线高度代表概率密度，曲线上方展示了后验分布的平均值

In [11]:
# 参数的后验分布图，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.plot_posterior(trace,var_names=['alpha','beta','sigma'])
# beta1和beta2的后验分布值大于0的比例，若大于95%则可认为对应的自变量对因变量有影响
print(f"beta0大于0的概率为{(trace.posterior.beta[0] > 0).mean().values}")
print(f"beta1大于0的概率为{(trace.posterior.beta[1] > 0).mean().values}")

beta0大于0的概率为0.99675
beta1大于0的概率为0.99725


<Figure size 1987.2x331.2 with 4 Axes>

#### 模型诊断

通过模型思维进行数据分析需要注意模型检验，即检验模型是否能有效的反应数据的特征。

下表格为模型参数的基本信息：

`mean`和`sd` 为各参数的均值和标准差；
hdi 3%-97% 为参数分布的可信区间的下限和上限
msce mean和sd 为mcmc采样标准误统计量的均值和标准差；
由于采样过程中存在自相关，不是所有样本都是有效的，ess bulk和tail 反应了mcmc采样有效样本数量相关性能，ess_bulk表明有效样本数的大小。
r_hat, $\hat{R}$, 为参数收敛性的指标。

In [12]:
# 参数的后验分布图，选取对象为trace，选取其中'alpha','beta','sigma'三个参数
az.summary(trace,var_names=['alpha','beta','sigma'])

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,0.000,0.019,-0.037,0.036,0.0,0.0,4051.0,3262.0,1.0
beta[0],0.049,0.021,0.010,0.088,0.0,0.0,3096.0,2762.0,1.0
beta[1],0.775,0.021,0.735,0.812,0.0,0.0,3594.0,3122.0,1.0
sigma,0.609,0.014,0.585,0.637,0.0,0.0,3625.0,2887.0,1.0


### 后验预测检验 ppc (posterior predictive check)

后验预测分布就是利用现有数据对未来数据进行预测，即

$p(\tilde{Y}|Y)=\int_{\Theta }^{}p(\tilde{Y}|\theta)p(\theta|Y)d\theta$

In [13]:
# 后验预测分布的计算仍在容器中进行
with linear_model:
    # pm.sample_posterior_predictive()利用trace.posterior的后验分布计算后验预测分布
    ppc_y = pm.sample_posterior_predictive(trace.posterior) 
#将ppc_y转化为InferenceData对象合并到trace中
az.concat(trace, az.from_pymc3(posterior_predictive=ppc_y), inplace=True)

/opt/conda/lib/python3.7/site-packages/arviz/data/io_pymc3.py:100: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  FutureWarning,


后验预测分布展示如下，横轴为观察到的数据，纵轴为概率密度，

黑色实线为真实的观测值的分布，蓝色实线为根据不同后验分布的参数值模拟得到的预测分布，

蓝色虚线为不同后验分布的参数值模拟得到的预测分布的平均值。

In [14]:
# 绘制后验预测分布
az.plot_ppc(trace)

<AxesSubplot:xlabel='y_obs'>

/opt/conda/lib/python3.7/site-packages/IPython/core/pylabtools.py:151: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  fig.canvas.print_figure(bytes_io, **kw)


<Figure size 432x288 with 1 Axes>

# 练习

还有研究发现个体创新行为可能与自尊水平有关。
SES_t:自尊水平；
EIB_t创新行为

In [15]:
data['SES_t'] = (data['SES_t'] - data['SES_t'].mean()) / data['SES_t'].std()#将变量进行标准化

data['EIB_t'] = (data['EIB_t'] - data['EIB_t'].mean()) / data['EIB_t'].std()#将变量进行标准化

plt.scatter(data['SES_t'],data['EIB_t'])
plt.xlabel('SES_t')
plt.ylabel('EIB_t')

Text(0, 0.5, 'EIB_t')

<Figure size 432x288 with 1 Axes>

## Workflow

尝试构建一个简单的线性模型

线性模型可以用概率的形式进行表达


$\alpha \sim Normal(0,10)$      ->      a $\sim$ Normal(mu,sigma)

$\beta  \sim Normal(0,10)$      ->      b $\sim$ Normal(mu,sigma)

$\sigma \sim HalfNormal(1)$      ->      sigma $\sim$ HalfNormal(1)

$\mu_i  = \alpha + \beta *x$      ->      mu = alpha + beta*x 

$y \sim Normal(\mu_i,sigma)$      ->      y $\sim$ Normal(mu,sigma)

请同学们从之前的代码中找到对应的部分，自己完成一个完整的workflow

### 建立模型

In [16]:
########################################################
# 练习
# 请尝试表达自变量与预测值的线性关系,并将...替换为表达式
########################################################
linear_model = pm.Model()
with linear_model :
    #
    alpha = pm.Normal('alpha',mu=0,sd=10)
    beta = pm.Normal('beta',mu=0,sd=10,shape=2)
    sigma = pm.HalfNormal('sigma',sd=1)
    #
    x = pm.Data("x", x)
    #
    mu = pm.Deterministic("mu", ...) 
    #
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma,observed=y)

NameError: name 'x' is not defined

In [ ]:
########################################################
# 模型结构
# 
########################################################

### 模型拟合

In [ ]:
########################################################
# 模型拟合
# 采样方法为mcmc，采样次数为3000次，tune次数为500，接受率为0.5
########################################################

### 参数的后验分布

In [ ]:
########################################################
# 绘制参数的采样情况
# 
########################################################

In [ ]:
########################################################
# 绘制参数的森林图
# 
########################################################

In [ ]:
########################################################
# 绘制参数的后验分布
# 
########################################################

In [ ]:
########################################################
# 判断自变量是否能影响因变量
# 
########################################################

### 模型诊断

In [ ]:
########################################################
# 描述模型的统计指标
# 
########################################################

In [ ]:
########################################################
# 绘制后验分布预测
# 
########################################################